In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from utilities import *
import time
data = FashionMNISTLoader(resize=(128,128),batch_size=32)

# 定义 ViT 模型
model = ViT(image_size=128, patch_size=16, num_classes=10, dim=1024, depth=2, heads=8, mlp_dim=4096, channels=1, dropout=0.1)

# 定义优化器函数和超参数
trainer_fn = torch.optim.SGD
hyperparams = {'lr': 0.1}

# 创建 Trainer 实例并训练模型
trainer = Trainer(trainer_fn, hyperparams, data, model, feature_dim=10, num_epochs=4, cuda=True)
trainer.fit()

Loss: 12.223644256591797
predicted: tensor([5, 0, 0, 0, 5], device='cuda:0')


KeyboardInterrupt: 